<a href="https://colab.research.google.com/github/KSNTrader/Trader/blob/main/2022_12_26_Selecionando_Jogos_para_suas_Estrat%C3%A9gias_(Atualizado)_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import datetime
from datetime import date
from datetime import timedelta

Date = date.today() + timedelta(days=1)
print("Data de Análise:",Date)

# Importando as Bibliotecas

In [ ]:
import pandas as pd
import numpy as np
from google.colab import data_table
data_table.enable_dataframe_formatter()
import warnings
warnings.filterwarnings('ignore')

# Importando a Base de Dados

In [ ]:
df = pd.read_excel("https://github.com/KSNTrader/Trader/blob/main/Argentina%20jogos%20Hoje.xlsx")

In [ ]:
# Resultado
df.loc[df['FT_Goals_H'] > df['FT_Goals_A'], 'FT_Result'] = "H"
df.loc[df['FT_Goals_H'] == df['FT_Goals_A'], 'FT_Result'] = "D"
df.loc[df['FT_Goals_H'] < df['FT_Goals_A'], 'FT_Result'] = "A"

# # Probabilidades
df['p_H'] = 1 / df.FT_Odds_H
df['p_D'] = 1 / df.FT_Odds_D
df['p_A'] = 1 / df.FT_Odds_A

# Valor do Gol
df['VG_H'] = df.FT_Goals_H * df.p_A
df['VG_A'] = df.FT_Goals_A * df.p_H
df['Media_VG_H'] = df.groupby('Home')['VG_H'].rolling(window=5, min_periods=2).mean().reset_index(0,drop=True)
df['Media_VG_A'] = df.groupby('Away')['VG_A'].rolling(window=5, min_periods=2).mean().reset_index(0,drop=True)
df['DesvPad_VG_H'] = df.groupby('Home')['VG_H'].rolling(window=5, min_periods=2).std().reset_index(0,drop=True)
df['DesvPad_VG_A'] = df.groupby('Away')['VG_A'].rolling(window=5, min_periods=2).std().reset_index(0,drop=True)
df['CV_VG_H'] = df['DesvPad_VG_H'] / df['Media_VG_H']
df['CV_VG_A'] = df['DesvPad_VG_A'] / df['Media_VG_A']

# Custo do Gol
df['CG_H'] = df.p_H / df.FT_Goals_H
df['CG_A'] = df.p_A / df.FT_Goals_A
df['Media_CG_H'] = df.groupby('Home')['CG_H'].rolling(window=5, min_periods=2).mean().reset_index(0,drop=True)
df['Media_CG_A'] = df.groupby('Away')['CG_A'].rolling(window=5, min_periods=2).mean().reset_index(0,drop=True)
df['DesvPad_CG_H'] = df.groupby('Home')['CG_H'].rolling(window=5, min_periods=2).std().reset_index(0,drop=True)
df['DesvPad_CG_A'] = df.groupby('Away')['CG_A'].rolling(window=5, min_periods=2).std().reset_index(0,drop=True)
df['CV_CG_H'] = df['DesvPad_CG_H'] / df['Media_CG_H']
df['CV_CG_A'] = df['DesvPad_CG_A'] / df['Media_CG_A']

# Arredondando as Variáveis
df['Media_VG_H'] = round(df['Media_VG_H'], 4)
df['Media_VG_A'] = round(df['Media_VG_A'], 4)
df['CV_VG_H'] = round(df['CV_VG_H'], 4)
df['CV_VG_A'] = round(df['CV_VG_A'], 4)
df['Media_CG_H'] = round(df['Media_CG_H'], 4)
df['Media_CG_A'] = round(df['Media_CG_A'], 4)
df['CV_CG_H'] = round(df['CV_CG_H'], 4)
df['CV_CG_A'] = round(df['CV_CG_A'], 4)

df.replace(np.inf, 1, inplace=True)
df.dropna(inplace=True)
df.reset_index(inplace=True, drop=True)
df.index = df.index.set_names(['Nº'])
df = df.rename(index=lambda x: x + 1)

# Importando os Jogos do Dia

In [ ]:
jogos_do_dia = pd.read_excel('https://github.com/futpythontrader/YouTube/blob/main/Jogos_do_Dia/'+str(Date)+'_FutPythonTrader_Jogos_do_Dia.xlsx?raw=true')

# Concatenando os Jogos do Dia com a Base de Dados

In [ ]:
df_H = df[['Home','Media_VG_H','CV_VG_H','Media_CG_H','CV_CG_H']]
df_A = df[['Away','Media_VG_A','CV_VG_A','Media_CG_A','CV_CG_A']]

In [ ]:
lista = []

for a,b,c,d,e,f,g,h in zip(jogos_do_dia.League,jogos_do_dia.Date,jogos_do_dia.Home,jogos_do_dia.Away,jogos_do_dia.FT_Odds_H,jogos_do_dia.FT_Odds_D,jogos_do_dia.FT_Odds_A,jogos_do_dia.FT_Odds_Over25):
    try:
        League = a
        Date = b
        home = c
        away = d
        FT_Odd_H = e
        FT_Odd_D = f
        FT_Odd_A = g
        FT_Odd_Over25 = h

        df1 = df_H[df_H.Home == home].tail(1)

        df2 = df_A[df_A.Away == away].tail(1)

        jogo = {}

        jogo["League"] = League
        jogo["Date"] = Date
        jogo["Home"] = home
        jogo["Away"] = away
        jogo["FT_Odd_H"] = FT_Odd_H
        jogo["FT_Odd_D"] = FT_Odd_D
        jogo["FT_Odd_A"] = FT_Odd_A
        jogo["FT_Odd_Over25"] = FT_Odd_Over25
        jogo["Media_VG_H"] = df1[df1.Home == home]['Media_VG_H'].iloc[0]
        jogo["CV_VG_H"] = df1[df1.Home == home]['CV_VG_H'].iloc[0]
        jogo["Media_CG_H"] = df1[df1.Home == home]['Media_CG_H'].iloc[0]
        jogo["CV_CG_H"] = df1[df1.Home == home]['CV_CG_H'].iloc[0]
        jogo["Media_VG_A"] = df2[df2.Away == away]['Media_VG_A'].iloc[0]
        jogo["CV_VG_A"] = df2[df2.Away == away]['CV_VG_A'].iloc[0]
        jogo["Media_CG_A"] = df2[df2.Away == away]['Media_CG_A'].iloc[0]
        jogo["CV_CG_A"] = df2[df2.Away == away]['CV_CG_A'].iloc[0]

        lista.append(jogo)

    except:
        pass

df = pd.DataFrame(lista)
df.reset_index(inplace=True, drop=True)
df.index = df.index.set_names(['Nº'])
df = df.rename(index=lambda x: x + 1)
df

# Filtro de Entradas

In [ ]:
filtro = (
# Valor do Gol do Mandante
((df.Media_VG_H >= 0.1) & (df.Media_VG_H <= 0.9))
# CV da Média do Valor do Gol do Mandante
& ((df.CV_VG_H >= 0.1) & (df.CV_VG_H <= 0.9))
# Média do Custo do Gol do Mandante
& ((df.Media_CG_H >= 0.1) & (df.Media_CG_H <= 0.9))
# CV da Média do Custo do Gol do Mandante
& ((df.CV_CG_H >= 0.1) & (df.CV_CG_H <= 0.9))
# Valor do Gol do Visitante
& ((df.Media_VG_A >= 0.1) & (df.Media_VG_A <= 0.9))
# CV da Média do Valor do Gol do Visitante
& ((df.CV_VG_A >= 0.1) & (df.CV_VG_A <= 0.9))
# Média do Custo do Gol do Visitante
& ((df.Media_CG_A >= 0.1) & (df.Media_CG_A <= 0.9))
# CV da Média do Custo do Gol do Visitante
& ((df.CV_CG_A >= 0.1) & (df.CV_CG_A <= 0.9))
)

Entradas = df[filtro]
Entradas.reset_index(inplace=True, drop=True)
Entradas.index = Entradas.index.set_names(['Nº'])
Entradas = Entradas.rename(index=lambda x: x + 1)

# Entradas

In [ ]:
Entradas